![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Lecture11_Homework_Xiaohan Cheng

## 3) Rank stocks in 1) by the autocorrelation using daily close data from 1/1/2021 to 3/1/2021

In [29]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/research/overview]
qb = QuantBook()

In [30]:
import math
start_time = datetime(2021, 1, 1) # start datetime for history call
end_time = datetime(2021, 3, 1) # end datetime for history call

In [3]:
universe = ["AMZN", "BABA", "HD", "TOYOY", "MCD", "NKE", "SBUX", "LOW", "PCLN", "TJX"]
log = pd.DataFrame()
log_diff = pd.DataFrame()
rho = pd.DataFrame()
for symbol in universe:
    qb.symbol = qb.AddEquity(symbol, Resolution.Daily)
    history = qb.History(qb.symbol.Symbol, start_time, end_time, Resolution.Daily)
    # Can also write like this: log[symbol] = history.loc[symbol].close.apply(lambda x: math.log(x))
    log[symbol] = np.log(history.loc[symbol].close)
    log_diff[symbol] = np.diff(log[symbol])
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.autocorr.html
    rho[symbol] = pd.Series(log_diff[symbol].autocorr(1))

In [4]:
rho


,NB,CMB,TRV,NOB,V,BRKB,MA,PYPL,MHP,GS
0,0.182211,0.285993,-0.081161,0.067906,-0.168926,-0.036506,-0.101583,0.029922,-0.19211,0.240589


In [5]:
rho.sort_values(by = 0, axis = 1, ascending = False)


,CMB,GS,NB,NOB,PYPL,BRKB,TRV,MA,V,MHP
0,0.285993,0.240589,0.182211,0.067906,0.029922,-0.036506,-0.081161,-0.101583,-0.168926,-0.19211


## 3) Rank stocks in 2) by the autocorrelation using daily close data from 1/1/2021 to 3/1/2021

In [31]:
universe = ["NB", "CMB", "TRV", "NOB", "V", "BRKB", "MA", "PYPL", "MHP", "GS"]
log = pd.DataFrame()
log_diff = pd.DataFrame()
rho = pd.DataFrame()
for symbol in universe:
    qb.symbol = qb.AddEquity(symbol, Resolution.Daily)
    history = qb.History(qb.symbol.Symbol, start_time, end_time, Resolution.Daily)
    # Can also write like this: log[symbol] = history.loc[symbol].close.apply(lambda x: math.log(x))
    log[symbol] = np.log(history.loc[symbol].close)
    log_diff[symbol] = np.diff(log[symbol])
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.autocorr.html
    rho[symbol] = pd.Series(log_diff[symbol].autocorr(1))
rho.sort_values(by = 0, axis = 1, ascending = False)

,CMB,GS,NB,NOB,PYPL,BRKB,TRV,MA,V,MHP
0,0.285993,0.240589,0.182211,0.067906,0.029922,-0.036506,-0.081161,-0.101583,-0.168926,-0.19211


## 4)Rank the stock pairs within 1)  by ADF then by Bruesch-Pagan test using the historical data from 9/1/2020 to 3/1/2021

In [32]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels import regression,stats
start_time = datetime(2020, 9, 1) # start datetime for history call
end_time = datetime(2021, 3, 1) # end datetime for history call

In [36]:
universe = ["AMZN", "BABA", "HD", "TOYOY", "MCD", "NKE", "SBUX", "LOW", "PCLN", "TJX"]
log = pd.DataFrame()
for symbol in universe:
    qb.symbol = qb.AddEquity(symbol, Resolution.Daily)
    history = qb.History(qb.symbol.Symbol, start_time, end_time, Resolution.Daily)
    # Can also write like this: log[symbol] = history.loc[symbol].close.apply(lambda x: math.log(x))
    log[symbol] = np.log(history.loc[symbol].close)

In [40]:
# https://www.cnblogs.com/shirely-2014/p/10512924.html
# This function returns the p-value of the Augmented Dickey-Fuller test
def ADF(X, Y):
    X = sm.add_constant(X)
    model = sm.OLS(Y, X)
    results = model.fit()
    sigma = math.sqrt(results.mse_resid) # standard deviation of the residual
    slope = results.params[1]
    intercept = results.params[0]
    res = results.resid # regression residual mean of res = 0 by definition
    zscore = res/sigma
    adf = adfuller (res)  # ADF Test -- Augmented Dickey-Fuller test for unit root
    return adf[1]

In [41]:
rank = pd.DataFrame()
for Y in universe:
    for X in universe:
        if universe.index(X) > universe.index(Y):
            rank[Y + "-" + X] = pd.Series(ADF(log[X], log[Y]))

In [42]:
rank

,AMZN-BABA,AMZN-HD,AMZN-TOYOY,AMZN-MCD,AMZN-NKE,AMZN-SBUX,AMZN-LOW,AMZN-PCLN,AMZN-TJX,BABA-HD,...,NKE-SBUX,NKE-LOW,NKE-PCLN,NKE-TJX,SBUX-LOW,SBUX-PCLN,SBUX-TJX,LOW-PCLN,LOW-TJX,PCLN-TJX
0,0.000385,0.000302,0.000178,0.000271,0.00009,0.000155,0.000045,0.000251,0.000118,0.058318,...,0.031884,0.203252,0.032866,0.003383,0.85411,0.010868,0.003236,0.140175,0.133414,0.180712


In [44]:
rank.sort_values(by = 0, axis = 1, ascending = True)

,AMZN-LOW,AMZN-NKE,AMZN-TJX,AMZN-SBUX,AMZN-TOYOY,AMZN-PCLN,AMZN-MCD,AMZN-HD,AMZN-BABA,MCD-PCLN,...,BABA-SBUX,LOW-TJX,BABA-NKE,LOW-PCLN,PCLN-TJX,NKE-LOW,MCD-LOW,BABA-LOW,TOYOY-LOW,SBUX-LOW
0,0.000045,0.00009,0.000118,0.000155,0.000178,0.000251,0.000271,0.000302,0.000385,0.000401,...,0.095436,0.133414,0.137115,0.140175,0.180712,0.203252,0.304103,0.544346,0.583061,0.85411


In [45]:
# This function returns the p-value of the Bruesch-Pagan test
def BP(X, Y):
    X = sm.add_constant(X)
    model = sm.OLS(Y, X)
    results = model.fit()
    sigma = math.sqrt(results.mse_resid) # standard deviation of the residual
    slope = results.params[1]
    intercept = results.params[0]
    res = results.resid # regression residual mean of res = 0 by definition
    zscore = res/sigma
    # p_value of Bruesch-Pagan test for heteroskedasticity
    bp = statsmodels.stats.diagnostic.het_breuschpagan(res, X)[3] 
    return bp

In [46]:
rank = pd.DataFrame()
for Y in universe:
    for X in universe:
        if universe.index(X) > universe.index(Y):
            rank[Y + "-" + X] = pd.Series(BP(log[X], log[Y]))

In [47]:
rank

,AMZN-BABA,AMZN-HD,AMZN-TOYOY,AMZN-MCD,AMZN-NKE,AMZN-SBUX,AMZN-LOW,AMZN-PCLN,AMZN-TJX,BABA-HD,...,NKE-SBUX,NKE-LOW,NKE-PCLN,NKE-TJX,SBUX-LOW,SBUX-PCLN,SBUX-TJX,LOW-PCLN,LOW-TJX,PCLN-TJX
0,0.008796,0.122931,0.001717,0.673388,0.000026,0.002009,0.952495,0.054052,0.003473,0.178933,...,4.721472e-08,0.051413,0.400429,2.743583e-08,0.780583,0.810531,0.000089,0.034884,0.07917,0.109935


In [48]:
rank.sort_values(by = 0, axis = 1, ascending = True)

,MCD-PCLN,NKE-TJX,NKE-SBUX,MCD-TJX,MCD-LOW,MCD-SBUX,TOYOY-LOW,AMZN-NKE,SBUX-TJX,BABA-TOYOY,...,BABA-SBUX,BABA-PCLN,AMZN-MCD,HD-TOYOY,SBUX-LOW,BABA-TJX,SBUX-PCLN,HD-SBUX,HD-NKE,AMZN-LOW
0,2.178093e-08,2.743583e-08,4.721472e-08,7.987606e-08,1.435214e-07,2.778005e-07,0.000005,0.000026,0.000089,0.001186,...,0.437142,0.469059,0.673388,0.760581,0.780583,0.78899,0.810531,0.881789,0.948104,0.952495


## 4)Rank the stock pairs within 2) by ADF then by Bruesch-Pagan test using the historical data from 9/1/2020 to 3/1/2021

In [49]:
universe = ["NB", "CMB", "TRV", "NOB", "V", "BRKB", "MA", "PYPL", "MHP", "GS"]
log = pd.DataFrame()
for symbol in universe:
    qb.symbol = qb.AddEquity(symbol, Resolution.Daily)
    history = qb.History(qb.symbol.Symbol, start_time, end_time, Resolution.Daily)
    # Can also write like this: log[symbol] = history.loc[symbol].close.apply(lambda x: math.log(x))
    log[symbol] = np.log(history.loc[symbol].close)
rankadf = pd.DataFrame()
rankbp = pd.DataFrame()
for Y in universe:
    for X in universe:
        if universe.index(X) > universe.index(Y):
            rankadf[Y + "-" + X] = pd.Series(ADF(log[X], log[Y]))
            rankbp[Y + "-" + X] = pd.Series(BP(log[X], log[Y]))

In [50]:
rankadf.sort_values(by = 0, axis = 1, ascending = True)

,PYPL-GS,V-BRKB,MA-GS,V-GS,NB-GS,MHP-GS,MA-PYPL,BRKB-PYPL,V-PYPL,TRV-BRKB,...,CMB-V,BRKB-MHP,NOB-MA,BRKB-MA,NB-TRV,V-MA,NOB-MHP,NB-MA,TRV-MA,CMB-MA
0,0.003392,0.008324,0.008578,0.012347,0.014754,0.015582,0.016473,0.01664,0.019429,0.028343,...,0.391925,0.406234,0.471284,0.485611,0.48917,0.499718,0.51297,0.624145,0.670625,0.687708


In [51]:
rankbp.sort_values(by = 0, axis = 1, ascending = True)

,TRV-BRKB,TRV-NOB,TRV-GS,CMB-NOB,NB-CMB,TRV-PYPL,BRKB-PYPL,CMB-BRKB,CMB-GS,NB-NOB,...,NB-MHP,BRKB-MHP,CMB-MHP,V-MA,CMB-V,NB-TRV,PYPL-MHP,NB-V,NOB-PYPL,MHP-GS
0,2.155300e-19,1.249350e-17,3.365456e-13,1.266615e-09,3.991781e-08,4.116896e-07,0.000002,0.000066,0.000145,0.000206,...,0.427577,0.485111,0.521998,0.527598,0.614835,0.717765,0.752811,0.91369,0.921253,0.965956


## Scratchwork

In [69]:
log

In [70]:
log.shape

In [71]:
log_diff

In [95]:
pd.Series(log_diff["AMZN"].autocorr(1))

In [96]:
rho["AMZN"] = pd.Series(log_diff["AMZN"].autocorr(1))

In [97]:
rho

In [73]:
log_diff["AMZN"]